In [1]:
import torch
import torchvision
import torch.nn as nn
from model1 import CBAM_ResNet18
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F


In [2]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
train_set = torchvision.datasets.CIFAR10(
    root='../data/CIFAR10/'
    ,train=True
    ,download=False
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip()
        
    ])
)

test_set = torchvision.datasets.CIFAR10(
    root='../data/CIFAR10/'
    ,train=False
    ,download=False
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip()
        
    ])
)

In [4]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

import torchvision 
import torchvision.transforms as transforms

In [5]:
class ChannelAttention(nn.Module):
    def __init__(self,in_channel):
        super(ChannelAttention,self).__init__()
        self.maxpool = nn.MaxPool2d(1)
        self.avgpool = nn.AvgPool2d(1)
        #5//2=2（2.5向负无穷方向取整为2），同时-5//2=-3（-2.5向负无穷方向取整为-3）
        #经过pool之后  维度编程 channel * 1*1大小
        #对于channel*1*1来说 卷积 和 mlp相同
        self.fc1 = nn.Conv2d(in_channels=in_channel,out_channels=in_channel//16,kernel_size=1,bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_channel//16,in_channel,kernel_size=1,bias=False)
        self.sigmod = nn.Sigmoid()

    def forward(self,t):
        max_out = self.maxpool(t)
        max_out = self.fc2(self.relu(self.fc1(max_out)))

        avg_out = self.avgpool(t)
        avg_out = self.fc2(self.relu(self.fc1(avg_out)))

        out = self.sigmod(max_out+avg_out)

        return out


In [6]:
class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention,self).__init__()
        self.conv1 = nn.Conv2d(2,1,kernel_size=7,padding=3,bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self,t):
        #c*H*W
        max_out,_= torch.max(t,dim=1,keepdim=True)
        avg_out = torch.mean(t,dim=1,keepdim=True)

        out = torch.cat([max_out,avg_out],dim=1)
        out = self.sigmoid(self.conv1(out))

        return out


In [7]:
class BasicBlock(nn.Module):
    expansion=1
    def __init__(self,in_channel,out_channel,stride=1,downsample=None,useCBAM=1):
        super(BasicBlock,self).__init__()
        self.useCBAM = useCBAM
        self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel
                                ,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel,
                                kernel_size=3,stride=1,padding=1,bias = False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.channel_atten= ChannelAttention(self.expansion*out_channel)
        self.spatial_atten = SpatialAttention()
        self.downsample = downsample

    def forward(self,t):
        #t 输入
        identity = t
        if self.downsample is not None:
            identity = self.downsample(t)
        out = self.relu(self.bn1(self.conv1(t)))
        out = self.bn2(self.conv2(out))
        if self.useCBAM==1:
            CBAM_Channel = out * self.channel_atten(out)
            out = CBAM_Channel*self.spatial_atten(CBAM_Channel)
        out = self.relu(out+identity)

        return out


In [8]:
class Bottlenect(nn.Module):
    expansion = 4

    def __init__(self,in_channel,out_channel,stride=1,downsample=None,useCBAM=1):
        super(Bottlenect,self).__init__()
        self.useCBAM = useCBAM
        #conv1没有padding 因为 维持原型 只改变通道数 第一层中接到池化后面 所以相当于通道数没有改变
        self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,
                                kernel_size=1,stride=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel,
                                kernel_size=3,stride=stride,padding = 1,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.conv3 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel*self.expansion,
                                kernel_size=1,stride=1,bias=False)
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
        self.channel_atten= ChannelAttention(self.expansion*out_channel)
        self.spatial_atten = SpatialAttention()
        self.downsample = downsample

    def forward(self,t):
        identity = t
        if self.downsample is not None:
            identity = self.downsample(t)
        
        out = self.relu(self.bn1(self.conv1(t)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        if self.useCBAM==1:
            print('useCBAM')
            CBAM_Channel = out * self.channel_atten(out)
            out = CBAM_Channel*self.spatial_atten(CBAM_Channel)

        out = self.relu(out+identity)
        return out

In [9]:
class ResNet(nn.Module):
    def __init__(self,block,block_num,num_classes=1000,useCBAM=None):
        super(ResNet,self).__init__()
        self.in_channel=64
        if useCBAM == None:
            self.useCBAM = []
            for i in range (4):
                self.useCBAM.append(None)
        else:
            self.useCBAM=useCBAM
            
        #o=112 (224-7+2p)/2 +1 =112   so p = 3
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=self.in_channel,kernel_size=7,
                                stride=2,padding=3,bias=False)
        #o=56  (112-3+2p)/2+1 = 56 so p=1  在pytorch中所有计算结果都是向下取整
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.layer1 = self._make_layer(block,64,block_num[0],stride=1,useCBAM=self.useCBAM[0])
        self.layer2 = self._make_layer(block,128,block_num[1],stride=2,useCBAM=self.useCBAM[1])
        self.layer3 = self._make_layer(block,256,block_num[2],stride=2,useCBAM=self.useCBAM[2])
        self.layer4 = self._make_layer(block,512,block_num[3],stride=2,useCBAM=self.useCBAM[3])

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*block.expansion,num_classes)

        #初始化
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')

    def _make_layer(self,block,channel,block_num,stride=1,useCBAM=None):
        downsample =None
        if stride != 1 or self.in_channel!=block.expansion * channel:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_channel,out_channels=block.expansion*channel,kernel_size=1,
                        stride=stride,bias=False),
                nn.BatchNorm2d(block.expansion*channel)
            )

        layers =[]
        if useCBAM == None:
            useCBAM =[]
            for i in range(block_num):
                useCBAM.append(1)

        layers.append(block(self.in_channel,channel,stride,downsample,useCBAM[0]))

        self.in_channel = block.expansion * channel

        for _ in range(1,block_num):
            layers.append(block(in_channel=self.in_channel,out_channel=channel,useCBAM=useCBAM[_]))

        return nn.Sequential(*layers)

    def forward(self,t):
        out = self.relu(self.bn1(self.conv1(t)))
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out,1)
        out = self.fc(out)

        return out

In [10]:
def CB_resnet18(num_classes =1000,useCBAM=None):
    return ResNet(BasicBlock,[2,2,2,2],num_classes,useCBAM)

def CB_resent34(num_classes = 1000,useCBAM=None):
    return ResNet(BasicBlock,[3,4,6,3],num_classes,useCBAM)

def CB_resnet50(num_classes=1000,useCBAM=None):
    return ResNet(Bottlenect,[3,4,6,3],num_classes,useCBAM)

def CB_resnet101(num_classes=1000,useCBAM=None):
    return ResNet(Bottlenect,[3,4,23,3],num_classes,useCBAM)

def CB_resnet152(num_classes=1000,useCBAM=None):
    return ResNet(Bottlenect,[3,8,36,3],num_classes,useCBAM)

In [11]:
useCBAM = []
useCBAM.append([1,0])
useCBAM.append([1,0])
useCBAM.append([1,0])
useCBAM.append([1,0])

#34
# useCBAM = []
# useCBAM.append([1,0,0])
# useCBAM.append([1,0,0,0])
# useCBAM.append([1,0,0,0,0,0])
# useCBAM.append([1,0,0])
#50
# useCBAM = []
# useCBAM.append([1,0,0])
# useCBAM.append([1,0,0,0])
# useCBAM.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
# useCBAM.append([1,0,0])
#101
# useCBAM = []
# useCBAM.append([1,0])
# useCBAM.append([1,0])
# useCBAM.append([1,0])
# useCBAM.append([1,0])
#152
# useCBAM = []
# useCBAM.append([1,0])
# useCBAM.append([1,0])
# useCBAM.append([1,0])
# useCBAM.append([1,0])


In [11]:
network = CB_resnet50(num_classes=10).to('cuda')
network.train()

train_loader = torch.utils.data.DataLoader(train_set,batch_size = 6)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

batch = next(iter(train_loader))
images = batch[0].to('cuda')
labels = batch[1].to('cuda')

preds = network(images)
loss = F.cross_entropy(preds,labels)

loss.backward()#计算梯度
optimizer.step()#更新权重

#-----------------------
print('loss1:',loss.item())
preds = network(images)
loss = F.cross_entropy(preds,labels)
print('loss2:',loss.item())


useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
loss1: 2.3456342220306396
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
useCBAM
loss2: 1.953810691833496


In [12]:
useCBAM = []
useCBAM.append([1,1])
useCBAM.append([1,1])
useCBAM.append([1,1])
useCBAM.append([1,1])
network =  CB_resnet18(num_classes=10,useCBAM=useCBAM).to('cuda')


train_loader = torch.utils.data.DataLoader(train_set,batch_size=6)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=6)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

total_loss = 0
total_correct = 0

count = 0
network.train()
for batch in train_loader:
    images = batch[0].to('cuda')
    labels = batch[1].to('cuda')
    
    preds = network(images)
    loss = F.cross_entropy(preds,labels)
    #------------要将梯度归零 因为pytorch会累加梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds,labels)
    print('where',count,'loss:',loss.item())
    count +=1


    #-----------------eval
print("epoch:",0,"total_correct",total_correct," loss:",total_loss)
total_correct / len(train_set)




where 0 loss: 2.3730571269989014
where 1 loss: 2.39601731300354
where 2 loss: 2.2707226276397705
where 3 loss: 2.3900747299194336
where 4 loss: 2.3624887466430664
where 5 loss: 2.227642059326172
where 6 loss: 2.3966922760009766
where 7 loss: 2.296625852584839
where 8 loss: 2.202094554901123
where 9 loss: 2.3507473468780518
where 10 loss: 2.295591115951538
where 11 loss: 2.3667867183685303
where 12 loss: 2.291285753250122
where 13 loss: 2.190047264099121
where 14 loss: 2.2288882732391357
where 15 loss: 2.194340229034424
where 16 loss: 1.9867397546768188
where 17 loss: 2.1458067893981934
where 18 loss: 2.2172796726226807
where 19 loss: 2.134735345840454
where 20 loss: 2.2243058681488037
where 21 loss: 2.067641258239746
where 22 loss: 1.9232114553451538
where 23 loss: 1.8622061014175415
where 24 loss: 2.3338820934295654
where 25 loss: 2.262399196624756
where 26 loss: 2.1371843814849854
where 27 loss: 2.4661333560943604
where 28 loss: 2.0390713214874268
where 29 loss: 2.3258862495422363
wh

ValueError: Expected input batch_size (2) to match target batch_size (6).

In [14]:
total_loss_eval = 0
total_correct_eval = 0
network.eval()  # 验证阶段，需要使模型处于验证阶段，即调用model.eval()
with torch.no_grad():  # torch.no_grad：一个tensor（命名为x）的requires_grad = True，由x得到的新tensor（命名为w-标量）requires_grad也为False，且grad_fn也为None,即不会对w求导
    for image, labels in test_loader:  # 测试集下载生成器
        # data是测试集的图像数据，target是测试集图片所代表的真实数字的值
        image = image.to('cuda')
        labels = labels.to('cuda')
        preds = network(image)
        loss = F.cross_entropy(preds,labels)
        total_loss_eval += loss.item()  # 计算测试集上的loss
        # 计算分类的准确率
        total_correct_eval += get_num_correct(preds,labels)  #  计算分类正确率
total_loss_eval /= len(test_loader.dataset)
print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        total_loss_eval, total_correct_eval, len(test_loader.dataset),
        100. * total_correct_eval / len(test_loader.dataset))) # print

AttributeError: 'float' object has no attribute 'append'

In [15]:
print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        total_loss_eval, total_correct_eval, len(test_loader.dataset),
        100. * total_correct_eval / len(test_loader.dataset))) # print


Test set: Avg. loss: 0.2473, Accuracy: 4692/10000 (47%)



In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')  # x轴是train_counter列表 y轴是train_losses列表 绘制曲线
plt.scatter(test_counter, test_losses, color='red')  # 绘制散点图
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')  # 给图加上图例
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
plt.show()


In [12]:
network =  CB_resnet18(num_classes=10,useCBAM=useCBAM).to('cuda')

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=1000)

optimizer = optim.Adam(network.parameters(),lr = 0.0001)

total_loss = 0
total_correct = 0

train_counter = []
test_counter = []

train_losses =[]
test_losses=[]

for epoch in range(5):
    total_loss = 0
    total_correct = 0
    train_counter.append(epoch)
    test_counter.append(epoch)
    for batch in train_loader:
        images = batch[0].to('cuda')
        labels = batch[1].to('cuda')
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        #------------要将梯度归零 因为pytorch会累加梯度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
    train_losses.append(total_correct/len(train_set))
    print("epoch:",epoch,"total_correct",total_correct," loss:",total_loss)

    total_loss_eval = 0
    total_correct_eval = 0
    network.eval()  # 验证阶段，需要使模型处于验证阶段，即调用model.eval()
    with torch.no_grad():  # torch.no_grad：一个tensor（命名为x）的requires_grad = True，由x得到的新tensor（命名为w-标量）requires_grad也为False，且grad_fn也为None,即不会对w求导
        for image, labels in test_loader:  # 测试集下载生成器
        # data是测试集的图像数据，target是测试集图片所代表的真实数字的值
            image = image.to('cuda')
            labels = labels.to('cuda')
            preds = network(image)
            loss = F.cross_entropy(preds,labels)
            total_loss_eval += loss.item()  # 计算测试集上的loss
        # 计算分类的准确率
            total_correct_eval += get_num_correct(preds,labels)  #  计算分类正确率
    total_loss_eval /= len(test_loader.dataset)
    test_losses.append(total_loss_eval)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        total_loss_eval, total_correct_eval, len(test_loader.dataset),
        100. * total_correct_eval / len(test_loader.dataset))) # print
print('train',total_correct/len(train_set))

epoch: 0 total_correct 19676  loss: 830.5627487897873


NameError: name 'test_loader' is not defined

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')  # x轴是train_counter列表 y轴是train_losses列表 绘制曲线
plt.scatter(test_counter, test_losses, color='red')  # 绘制散点图
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')  # 给图加上图例
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
plt.show()

In [ ]:
torch.save(network.state_dict(),'pt')